In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
from tqdm import tqdm
import random

In [2]:
links = pd.read_csv("./Data/Articles Links.csv")

In [3]:
links = links.drop("Unnamed: 0",axis=1)

In [13]:
links.shape

(1018, 2)

Scraping Articles

In [28]:
headers = []
authors = []
sources = []
published_dates = []
articles_body = []
urls = []
sub_headings = []

In [80]:

def scrape_article(soup,url):
    header = soup.find("div",{"id":"block-mit-page-title"}).find("span")
    if not header:
        header = ""
        headers.append(header)
    else:
        headers.append(header.get_text())

    author = soup.find("span",{"class":"news-article--author"})
    if not author:
        author = ""
        authors.append(author)
    else: 
        authors.append(author.get_text())

    source = soup.find("span",{"class":"news-article--source"})
    if not source:
        source = ""
        sources.append(source)
    else:
        sources.append(source.get_text())
        
    publish_date = soup.find("div",{"class":"news-article--publication-date"}).find("time")
    if not publish_date:
        publish_date = ""
        published_dates.append(publish_date)
    else:
        published_dates.append(publish_date.get_text())

    article_body_tag = soup.find("div",{"class":"paragraph paragraph--type--content-block-text paragraph--view-mode--default"})
    if not article_body_tag:
        article_body_tag = ""
        articles_body.append(body)
    else: 
        text_body = []
        for text in article_body_tag.find_all("p"):
            body = text.get_text().strip()
            text_body.append(body)
        articles_body.append(text_body)

    sub_heading = soup.find("div",{"class":"news-article--dek"})
    if not sub_heading:
        sub_heading = ""
        sub_headings.append(sub_heading)
    else:
        sub_headings.append(sub_heading.find("span").get_text())

    urls.append(url)


In [ ]:
for index,url in enumerate(links["Link"]):
    response = requests.get(url)
    status_code = response.status_code
    if status_code == 200:
        data = response.text
        soup = BeautifulSoup(data, "html.parser")

        # sleep(random.choice([0.3,0.5]))
        
        is_page_not_found = soup.find("body")["class"][0]
        if is_page_not_found == "page-not-found":
            continue
        else:
            scrape_article(soup,url)
            print(index)
        
    
    else:
        print("Response Code:",status_code)
        print("Breakin Code....")
        break

In [87]:
articles = pd.DataFrame({"Published Date":published_dates,"Author":authors,"Source":sources,"Article Header":headers,"Sub_Headings":sub_headings,"Article Body":articles_body,"Url":urls})

In [90]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018 entries, 0 to 1017
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Published Date  1018 non-null   object
 1   Author          1018 non-null   object
 2   Source          1018 non-null   object
 3   Article Header  1018 non-null   object
 4   Sub_Headings    1018 non-null   object
 5   Article Body    1018 non-null   object
 6   Url             1018 non-null   object
dtypes: object(7)
memory usage: 55.8+ KB


In [92]:
articles.to_csv("./Data/Articles.csv")